# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/bidding-3233000
INFO:tensorflow:Restoring parameters from ..\Models/binfo-286000
INFO:tensorflow:Restoring parameters from ..\Models/lead-1000000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = 'AQ8.KQ94.JT.KJ52'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["PAD_START", "PAD_START", "PAD_START", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

Sampling for aution: ['PAD_START', 'PAD_START', 'PAD_START', 'PASS'] trying to find 5000
sample_cards_auction, nsteps= 2
NS:  1.0 EW:  2.0 Auction:  ['PAD_START', 'PAD_START', 'PAD_START', 'PASS']
n_samples 5000 matching bidding info


[{'call': '1N', 'insta_score': 0.9994}]

In [4]:
auction = ["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

Sampling for aution: ['PAD_START', 'PAD_START', 'PAD_START', 'PASS', '1N', 'PASS', '3S', 'PASS'] trying to find 5000
sample_cards_auction, nsteps= 3
NS:  1.0 EW:  2.0 Auction:  ['PAD_START', 'PAD_START', 'PAD_START', 'PASS', '1N', 'PASS', '3S', 'PASS']
n_samples 3212 matching bidding info
 4H   0.7903 Samples: 23
['4HN', '4HN', '4HN', '4HN', '4HN', '4HN', '4HN', '4HN', '4HN', '4HN', '4HN', '4HN', '4HN', '4HN', '4HN', '4HN', '6HN', '4HN', '6HN', '6HN', '4HN', '4HN', '4HN']
[[0.   0.   0.   0.   0.   0.   0.   0.02 0.04 0.12 0.24 0.31 0.21 0.06]
 [0.   0.   0.   0.   0.   0.   0.01 0.02 0.04 0.12 0.25 0.3  0.2  0.06]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.04 0.16 0.34 0.33 0.11]
 [0.   0.   0.   0.   0.   0.   0.   0.01 0.03 0.1  0.22 0.31 0.23 0.08]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.05 0.18 0.36 0.31 0.09]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.03 0.14 0.35 0.36 0.11]
 [0.   0.   0.   0.   0.   0.   0.   0.01 0.03 0.1  0.23 0.33 0.23 0.07]
 [0.   0.   

[{'call': '4H',
  'insta_score': 0.7903,
  'expected_score': 473,
  'adjustment': 40},
 {'call': '3N', 'insta_score': 0.1586, 'expected_score': 415, 'adjustment': 8}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

['AQ8.KQ9x.JT.KJxx Jxxxx.JTx.AKx.9x x.A8xx.Q8xx.AT8x KT9x.xx.9xxx.Qxx 0.98702 P-1N-P-3S-P-4H-P-P-P (10.6)  P-1N-P-3S-P-3N-P-P-P (9.6) ',
 'AQ8.KQ9x.JT.KJxx KJTxx.Jx.9x.A9xx x.ATxx.Axxx.QT8x 9xxx.8xx.KQ8xx.x 0.98680 P-1N-P-3S-P-4H-P-P-P (10.6)  P-1N-P-3S-P-3N-P-P-P (9.0) ',
 'AQ8.KQ9x.JT.KJxx J9xxxx.Txx.Ax.Tx T.AJxx.Q8xx.AQxx Kxx.8x.K9xxx.98x 0.98504 P-1N-P-3S-P-4H-P-P-P (11.2)  P-1N-P-3S-P-3N-P-P-P (10.2) ',
 'AQ8.KQ9x.JT.KJxx T9xxx.8xx.K8xxx. J.ATxx.AQ9x.QTxx Kxxx.Jx.xx.A98xx 0.98215 P-1N-P-3S-P-4H-P-P-P (10.8)  P-1N-P-3S-P-3N-P-P-P (10.4) ',
 'AQ8.KQ9x.JT.KJxx KT9xxx.8xx.xx.9x x.AJxx.Q9xx.AQxx Jxx.Tx.AK8xx.T8x 0.98164 P-1N-P-3S-P-4H-P-P-P (11.2)  P-1N-P-3S-P-3N-P-P-P (9.9) ',
 'AQ8.KQ9x.JT.KJxx Kxxx.AJ.98xxx.8x 9.8xxx.AKQx.ATxx JTxxx.Txx.xx.Q9x 0.98071 P-1N-P-3S-P-4H-P-P-P (11.3)  P-1N-P-3S-P-3N-P-P-P (10.3) ',
 'AQ8.KQ9x.JT.KJxx JTxxx.xxx.Q8x.xx x.AT8x.A9xx.QT9x K9xx.Jx.Kxxx.A8x 0.97611 P-1N-P-3S-P-4H-P-P-P (10.8)  P-1N-P-3S-P-3N-P-P-P (9.3) ',
 'AQ8.KQ9x.JT.KJxx 9xxxx.xx.A9xx.Q9 x.

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [6]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

sample_cards_auction, nsteps= 3
NS:  1.0 EW:  2.0 Auction:  ['PAD_START', 'PAD_START', 'PAD_START', 'PASS', '1N', 'PASS', '3S', 'PASS']
n_samples 3172 matching bidding info
AQ8.KQ9x.JT.KJxx KT9xxx.8xx.xx.9x .AJTx.Q8xx.A8xxx Jxxx.xx.AK9xx.QT
AQ8.KQ9x.JT.KJxx K9xxxx.x.9xx.8xx x.Txxx.AQ8x.AT9x JTx.AJ8x.Kxxx.Qx
AQ8.KQ9x.JT.KJxx K9xxx.xx.xxx.Q9x T.A8xx.Q8xx.AT8x Jxxx.JTx.AK9x.xx
AQ8.KQ9x.JT.KJxx KTxxx.8xx.98xx.Q x.ATxx.AKxx.9xxx J9xx.Jx.Qxx.AT8x
AQ8.KQ9x.JT.KJxx Txxxx.A8x.xxx.Ax x.Jxxx.AKQ98.xxx KJ9x.Tx.xxx.QT98
AQ8.KQ9x.JT.KJxx KJxxxx.8x.xx.ATx 9.xxxx.AK8x.Q8xx Txx.AJT.Q9xxx.9x
AQ8.KQ9x.JT.KJxx Txxxx.J8x.Axx.xx x.Txxx.Kxx.AQTxx KJ9x.Ax.Q98xx.98
AQ8.KQ9x.JT.KJxx JTxxx.xxx.Kxx.Ax 9.AJTx.A8xxx.Txx Kxxx.8x.Q9x.Q98x
AQ8.KQ9x.JT.KJxx KJTxx.8xx.Q.Q8xx 9.AJxx.98xxx.Axx xxxx.Tx.AKxxx.T9
AQ8.KQ9x.JT.KJxx KTxxx.xxx.Qx.T98 9.JTxx.Axxxx.Axx Jxxx.A8.K98x.Qxx
AQ8.KQ9x.JT.KJxx KTxx.JTx.8xx.8xx x.A8xx.9xx.AQT9x J9xxx.xx.AKQxx.x
AQ8.KQ9x.JT.KJxx KJ9xxx.T.K8x.Txx x.AJ8x.A9xx.A8xx Txx.xxxx.Qxxx.Q9
AQ8.KQ9x.JT